# Logistic Regression and Survival Analysis - Climate Activism

### Features
In this notebook we firstly classify the activation of the user.
We perform a logistic regression, where the target variable is the activation of the user, i.e. user j write in one of the target subreddits in the period t, [t + 7days].

The indipendent variables are:
- sociodemographic features of the users: whether they are in the first/last quartile of the users with highest in male/female, rich/poor, left/right, old/young scores. Each of the first 10,000 subreddit has a score, and each user has a score that is the weighted average of the scores assigned to the subreddits where she commented.
- control variables of the activity: variables controlling the activity of the user, as average number of comments per week, average active days in Reddit per week, average length of discussion, average number of different subreddits she wrote in
- interaction variables: average number of different comments interacting with active authors per week, average number of different parent_id interacting with active authors per week, average number of different interacting active authors per week
- subreddit variables: one boolean for each of the 1000 most popular subreddit in our dataset, that is True if the author wrote in the subreddit, False otherwise
- news variables: proportion of news related to climate, climate change or natural disaster in the country the user is assigned to. The median for all non-georeferenced users

Control, interaction, subreddit and news variabls are replicated for three different time periods: week (t-1week, t), month (t-5weeks, t-1week), year(t-52weeks, t-5weeks).

### Logistic regression

We apply 1000 bootstrap sampling, then PCA, and keep the first 20 PCs.
Then logistic regression.
The coefficient are given by the average of the coefficients obtained at each sample.
The p-value is the proportion of experiments that gave the same sign.

Then, we repeat the experiment removing some of the variables.

### Survival analysis

Then, we perform survival analysis to infer the variables that influence the rate of activation, among the users that do activate (no right censoring).
Again, we apply bootstrap sampling, then PCA keeping the first 20 PCs.

In [1]:
import pandas as pd
import sys
sys.path += ["../src"]
import features_cox_week as ft
import climact_utils as cu
import cox_logreg_experiments as cle
import numpy as np
from importlib import reload
from glob import glob

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
from importlib import reload

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [4]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

### Create Dataframe

In [5]:
from create_features_df import create_features_df

In [6]:
subreddit_class = "activism"

In [7]:
df_lr = create_features_df(subreddit_class, months_list = cu.all_months[:-1],
                           balanced_classes = True, random_state = 100, verbose = True, )

Activity features
Subreddit features
News features
Merge
Sociodemographic features


In [8]:
df_lr.shape

(8226, 4026)

In [9]:
df_lr.activation.value_counts()

activation
True     4113
False    4113
Name: count, dtype: int64

### Exploratory Analysis

In [10]:
all_non_subreddit_features = [u for u in df_lr.columns if u[0] != "r"]

In [11]:
df_lr[all_non_subreddit_features].groupby("activation").mean().T.apply(lambda x: x[True] / x[False], axis = 1).replace(np.inf, np.nan).dropna().sort_values(ascending = False)

n_different_comments_with_active_link_id_link_id_short_long_ratio        14.795934
n_different_active_authors_link_id_link_id_short_long_ratio               8.874064
n_different_comments_with_active_id_parent_id_short                       5.296830
n_different_comments_with_active_parent_id_parent_id_short_long_ratio     5.169527
n_different_active_authors_id_parent_id_short                             5.070632
                                                                           ...    
sociodemo_young                                                           0.834244
n_different_comments_with_active_parent_id_parent_id_long                 0.775365
sociodemo_rich                                                            0.756228
sociodemo_right                                                           0.620339
duration                                                                  0.573891
Length: 73, dtype: float64

In [12]:
df_lr[all_non_subreddit_features].corr()["activation"].sort_values(ascending = False)

activation                                          1.000000
n_active_days_author_week_short                     0.217552
n_different_subreddits_short                        0.209334
n_different_active_authors_link_id_link_id_short    0.208146
n_different_active_authors_id_parent_id_short       0.194203
                                                      ...   
norm_climate_action_long                           -0.070142
sociodemo_rich                                     -0.077995
avg_comments_per_thread_long                       -0.083940
sociodemo_right                                    -0.128938
duration                                           -0.711426
Name: activation, Length: 74, dtype: float64

### Logistic regression - single experiment

In [13]:
pca = PCA(n_components = 20)
# get the z-score
scaler = StandardScaler()
logistic = LogisticRegression(max_iter=10000, tol=0.1)

In [14]:
np.random.seed(11)
msk = np.random.rand(len(df_lr)) < 0.75
data_train = df_lr.drop("duration", axis = 1)[msk]
data_test = df_lr.drop("duration", axis = 1)[~msk]

In [15]:
X_train, y_train = data_train.drop("activation", axis = 1), data_train["activation"]
X_test, y_test = data_test.drop("activation", axis = 1), data_test["activation"]

In [16]:
scaler.fit(X_train)
X_scale_train = scaler.transform(X_train)
# X_scale_train = X_train
# X_scale_train = scale_zscore(X_train)

In [17]:
pca.fit(X_scale_train)
X_pca_train = pca.transform(X_scale_train)

In [18]:
logistic.fit(X_pca_train, y_train)
logistic.coef_

array([[ 5.89435543e-02,  1.92646292e-01,  7.72099814e-03,
        -3.98240526e-02, -1.17494999e-01, -6.93373124e-02,
         2.55403072e-02, -1.24307988e-01, -1.83943660e-02,
        -8.19341065e-05,  6.28179481e-02, -1.69301304e-02,
         4.90360506e-02, -5.54376195e-02, -1.03580154e-03,
         3.51802514e-02, -1.61823832e-02,  5.63080857e-02,
         1.13722562e-01, -6.51897033e-02]])

In [19]:
X_scale_test = scaler.transform(X_test)
# X_scale_test = scale_zscore(X_test)
X_pca_test = pca.transform(X_scale_test)

In [20]:
cf = pd.DataFrame([logistic.predict(X_pca_test) + 0., y_test + 0.], index = ["predict", "actual"]).T.value_counts().unstack()
cf

actual,0.0,1.0
predict,,
0.0,781,404
1.0,199,645


In [21]:
1 - y_test.mean()

0.4829965500246427

In [22]:
cle.confusion_matrix_analysis(cf)

{'accuracy': 0.7028092656481025,
 'precision': 0.764218009478673,
 'recall': 0.6148713060057197,
 'random_baseline': 0.5170034499753573}

In [23]:
original_coef = np.dot(logistic.coef_, pca.components_)[0]

In [24]:
pd.Series(original_coef, index = X_train.columns).sort_values(ascending = False).head(10)

n_different_active_authors_parent_id_id_short           0.042926
n_different_active_authors_parent_id_id_medium          0.041014
n_different_comments_with_active_parent_id_id_short     0.040252
n_different_active_authors_id_parent_id_short           0.039995
n_different_active_authors_id_parent_id_medium          0.038738
n_different_comments_with_active_id_parent_id_medium    0.034993
n_different_comments_with_active_parent_id_id_medium    0.034795
n_different_comments_with_active_id_parent_id_short     0.033739
rChapoTrapHouse_short                                   0.026700
n_submissions_author_week_short                         0.026392
dtype: float64

### Logistic regression - Bootstrap sampling

In [25]:
def pca_log_reg(df_lr, features = None, test = False):
    pca = PCA(n_components = 8)
    scaler = StandardScaler()
    logistic = LogisticRegression(max_iter=10000, tol=0.1)
    if test:
        msk = np.random.rand(len(df_lr)) < 0.75
        data_train = df_lr.drop("duration", axis = 1)[msk]
        data_test = df_lr.drop("duration", axis = 1)[~msk]
        X, y = data_train.drop("activation", axis = 1), data_train["activation"]
        X_test, y_test = data_test.drop("activation", axis = 1), data_test["activation"]
    else:
        data_train = df_lr.drop("duration", axis = 1)
        X, y = data_train.drop("activation", axis = 1), data_train["activation"]
    if features is not None:
        X = X[features]
        X_test = X_test[features]
    scaler.fit(X)
    X_scale = scaler.transform(X)
    # X_scale = scale_zscore(X)
    pca.fit(X_scale)
    X_pca = pca.transform(X_scale)
    logistic.fit(X_pca, y)
    original_coef = np.dot(logistic.coef_, pca.components_)[0]
    if test:
        X_scale_test = scaler.transform(X_test)
        X_pca_test = pca.transform(X_scale_test)
        cf = pd.DataFrame([logistic.predict(X_pca_test) + 0., y_test + 0.], index = ["predict", "actual"]).T.value_counts().unstack()
        return pd.Series(original_coef, index = X.columns), cf
        
    else:
        return pd.Series(original_coef, index = X.columns)

In [26]:
n_bootstraps = 1000
coef_boot = pd.DataFrame([pca_log_reg(cle.get_bootstrap_df(df_lr, random_state = k*100)) for k in range(n_bootstraps)])

In [27]:
coef_means = coef_boot.mean()
coef_pvalues = [min(u, 1 - u) for u in (coef_boot > 0).sum() / n_bootstraps]

In [28]:
coef_logreg = pd.DataFrame({"coefficient": coef_means, "p": coef_pvalues, "exp_coefficient": np.exp(coef_means)}).rename_axis("variable")

In [29]:
coef_logreg.to_csv(cu.data_path + "experiments_logreg/lr_activism_1000boot_all_240611.csv")

In [30]:
coef_logreg.query("p < 0.05").sort_values("coefficient")

,coefficient,p,exp_coefficient
variable,,,
rnextfuckinglevel_long,-0.010460,0.0,0.989594
rTikTokCringe_long,-0.008657,0.0,0.991381
rHolUp_long,-0.008380,0.0,0.991655
rIdiotsInCars_long,-0.008253,0.0,0.991781
rThatsInsane_long,-0.008227,0.0,0.991807
...,...,...,...
n_comments_author_week_short,0.023740,0.0,1.024024
n_different_active_authors_parent_id_id_short,0.024266,0.0,1.024563
n_different_subreddits_short,0.024412,0.0,1.024713


### Log Odds analysis

In [31]:
# age "young - old"
# gender "male - female"
# partisan "left - right"
# affluence "poor - rich"

In [32]:
# left, poor, old, female are predicted as positive predictors
# note that among the negative predictor subreddits, some have very young scores (teenagers, RoastMe, dankmemes, AskOuija, greentext, PewdiepieSubmissions)
# on the other hand, collapse, worldnews and chomsky have old scores
coef_logreg.loc[["quartile" in u for u in coef_logreg.reset_index()["variable"]], :]

,coefficient,p,exp_coefficient
variable,,,


In [33]:
coef_logreg.loc[["norm" in u for u in coef_logreg.reset_index()["variable"]], :].sort_values("coefficient", ascending = False)

,coefficient,p,exp_coefficient
variable,,,
norm_climate_action_short_long_ratio,0.005586,0.000,1.005601
norm_natural_disaster_long,0.005371,0.000,1.005385
norm_climate_short_long_ratio,0.002694,0.004,1.002698
norm_climate_medium,0.001963,0.048,1.001965
norm_climate_short,0.001651,0.054,1.001652
norm_natural_disaster_short,0.001577,0.026,1.001578
norm_natural_disaster_medium,0.001417,0.075,1.001418
norm_climate_action_short,0.000629,0.281,1.000629
norm_natural_disaster_short_long_ratio,-0.000230,0.375,0.999770


In [ ]:
# interaction variables
coef_logreg.loc[[u in [u + v for u in ft.features["interaction"] for v in ["_short", "_medium", "_long", "_short_long_ratio"]]
                  for u in coef_logreg.reset_index()["variable"]], :].sort_values("coefficient", ascending = False)

,coefficient,p,exp_coefficient
variable,,,
n_different_parent_id_with_active_short,0.022630,0.000,1.022888
n_different_parent_id_with_active_medium,0.020193,0.000,1.020398
n_different_parent_id_with_active_short_long_ratio,0.013108,0.000,1.013195
n_different_active_authors_short,0.012840,0.000,1.012923
n_different_comments_with_active_short,0.010315,0.000,1.010368
n_different_active_authors_medium,0.010187,0.000,1.010239
n_different_active_authors_short_long_ratio,0.008859,0.000,1.008899
n_different_comments_with_active_short_long_ratio,0.008155,0.000,1.008189
n_different_parent_id_with_active_long,0.008069,0.000,1.008102


In [ ]:
# positive subreddits
coef_logreg.loc[[u[0] == "r" for u in coef_logreg.reset_index()["variable"]], :].sort_values("coefficient", ascending = False).head(20)

,coefficient,p,exp_coefficient
variable,,,
rCOMPLETEANARCHY_medium,0.028056,0.0,1.028453
rCOMPLETEANARCHY_long,0.027891,0.0,1.028284
rChapoTrapHouse_long,0.026853,0.0,1.027217
rChapoTrapHouse_medium,0.025926,0.0,1.026265
rAnarchism_long,0.025697,0.0,1.026030
rsocialism_long,0.025138,0.0,1.025457
rChapoTrapHouse_short,0.024346,0.0,1.024645
rCOMPLETEANARCHY_short,0.023440,0.0,1.023717
rAnarchism_medium,0.023051,0.0,1.023319


In [ ]:
# negative subreddits
coef_logreg.loc[[u[0] == "r" for u in coef_logreg.reset_index()["variable"]], :].sort_values("coefficient", ascending = True).head(20)

,coefficient,p,exp_coefficient
variable,,,
rpcmasterrace_long,-0.012970,0.0,0.987113
rgaming_long,-0.012400,0.0,0.987677
rxboxone_long,-0.010453,0.0,0.989601
rapexlegends_long,-0.010154,0.0,0.989897
rnextfuckinglevel_long,-0.009989,0.0,0.990060
rUnexpected_long,-0.009464,0.0,0.990581
rnevertellmetheodds_long,-0.009313,0.0,0.990730
rWhatcouldgowrong_long,-0.009085,0.0,0.990956
rPS4_long,-0.008829,0.0,0.991210


### Different features

In [ ]:
activity_and_news_features = ["_".join([u,p]) for u in ft.features["control"] + ft.features["interaction"] + ft.features["norm_news"] for p in ["week", "month", "year", "week_year_ratio"]]
activity_news_lr = pca_log_reg(df_lr, features = activity_and_news_features, test = True)

In [ ]:
activity_news_lr[0].sort_values(ascending = False)

n_different_subreddits_week_year_ratio               0.162181
n_active_days_author_week_week_year_ratio            0.137993
n_comments_author_week_week_year_ratio               0.132887
n_comments_author_week_week                          0.119238
n_different_parent_id_with_active_week_year_ratio    0.116453
n_different_comments_with_active_week_year_ratio     0.115519
n_different_active_authors_week_year_ratio           0.114413
n_different_subreddits_week                          0.098771
n_different_parent_id_with_active_week               0.088029
n_comments_author_week_month                         0.071273
n_different_subreddits_month                         0.060162
n_different_active_authors_week                      0.056658
n_active_days_author_week_week                       0.055213
avg_comments_per_thread_week_year_ratio              0.053368
n_different_parent_id_with_active_month              0.041450
norm_climate_month                                   0.023368
n_differ

In [ ]:
cle.confusion_matrix_analysis(activity_news_lr[1])

{'accuracy': 0.6430568499534017,
 'precision': 0.6536748329621381,
 'recall': 0.5633397312859885,
 'random_baseline': 0.5144454799627214}

In [ ]:
sociodemographic_features = [u for u in df_lr.columns if "quartile" in u]
sociodemographic_features_lr = pca_log_reg(df_lr, features = sociodemographic_features, test = True)

In [ ]:
sociodemographic_features_lr[0]

quartile1_affluence    0.202594
quartile1_age         -0.182440
quartile1_gender      -0.064025
quartile1_partisan     0.281238
quartile4_affluence   -0.243590
quartile4_age          0.152031
quartile4_gender      -0.147307
quartile4_partisan    -0.360296
dtype: float64

In [ ]:
cle.confusion_matrix_analysis(sociodemographic_features_lr[1])

{'accuracy': 0.6229353468617272,
 'precision': 0.6150159744408946,
 'recall': 0.7083716651333947,
 'random_baseline': 0.512977819726286}

### PCA

In [ ]:
pca_df = pd.DataFrame(pca.components_, columns = data_train.drop("activation", axis = 1).columns)

In [ ]:
pd.DataFrame([list(pca_df.loc[i].sort_values(ascending = False).head(15).index) for i in range(20)]).T

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,n_different_subreddits_long,rwallstreetbetsHUZZAH_short,rchess_short_long_ratio,rwallstreetbetsHUZZAH_short_long_ratio,rflorida_short_long_ratio,rnewjersey_short_long_ratio,rCringeAnarchy_long,rnba_long,rKitchenConfidential_short_long_ratio,rAstuff_short_long_ratio,rColumbus_short_long_ratio,rForwardsFromKlandma_short_long_ratio,rdestiny2_short_long_ratio,rColumbus_short_long_ratio,rForwardsFromKlandma_short_long_ratio,rForwardsFromKlandma_short_long_ratio,rswtor_short_long_ratio,rSmite_short,rPiracy_short_long_ratio,rcanada_medium
1,n_different_subreddits_medium,rMLPLounge_short,relderscrollsonline_short_long_ratio,rMLPLounge_short,rnewjersey_short_long_ratio,rflorida_short_long_ratio,rChapoTrapHouse_long,sociodemo_right,rAstuff_short_long_ratio,rAnythingGoesNews_short_long_ratio,rAstuff_short_long_ratio,rdestiny2_short_long_ratio,rForwardsFromKlandma_short_long_ratio,rAstuff_short_long_ratio,rdestiny2_short_long_ratio,rdestiny2_short_long_ratio,rdestiny2_short_long_ratio,rGuildwars2_short,rteenagersnew_short_long_ratio,rCanadaPolitics_long
2,n_different_subreddits_short,relderscrollsonline_short_long_ratio,rMLPLounge_short,relderscrollsonline_short_long_ratio,rnextfuckinglevel_long,rdogs_short_long_ratio,rChapoTrapHouse_medium,rnfl_long,rAnythingGoesNews_short_long_ratio,rColumbus_short_long_ratio,rAnythingGoesNews_short_long_ratio,resist_short_long_ratio,runitedkingdom_long,rAnythingGoesNews_short_long_ratio,rRight_Wing_Politics_short_long_ratio,rchildfree_long,rForwardsFromKlandma_short_long_ratio,rGuildwars2_short_long_ratio,rMujico_medium,ronguardforthee_long
3,n_different_active_authors_link_id_link_id_medium,rchess_short_long_ratio,rwallstreetbetsHUZZAH_short,rwallstreetbetsHUZZAH_short,rAtlanta_short_long_ratio,rnutrition_short_long_ratio,rMemeEconomy_long,rnfl_medium,rColumbus_short_long_ratio,rCoronavirusUS_short_long_ratio,rAstuff_short,rSandersForPresident_long,rukpolitics_long,rvancouver_short,rRight_Wing_Politics_short,rpovertyfinance_medium,rwestworld_short_long_ratio,rKitchenConfidential_short_long_ratio,rteenagersnew_short,rcanada_short
4,n_active_days_author_week_long,rwallstreetbetsHUZZAH_short_long_ratio,rwallstreetbetsHUZZAH_short_long_ratio,rchess_short_long_ratio,rdogs_short_long_ratio,rAtlanta_short_long_ratio,rComedyCemetery_long,rnba_medium,rcenterleftpolitics_short,rAstuff_short,rcenterleftpolitics_short_long_ratio,rJustrolledintotheshop_long,runitedkingdom_medium,rnvidia_long,rwoahdude_short_long_ratio,rchildfree_medium,rswtor_short,rSmite_medium,rPolcompball_medium,rCanadaPolitics_medium
5,n_active_days_author_week_medium,n_different_comments_with_active_id_parent_id_...,rMinneapolis_short_long_ratio,rCoronavirusDownunder_short,rnutrition_short_long_ratio,rSeattle_short_long_ratio,rFellowKids_long,rConservative_long,rcenterleftpolitics_short_long_ratio,rcenterleftpolitics_short_long_ratio,rCoronavirusUS_short_long_ratio,resist_short,rCasualUK_long,rGuildwars2_short,resist_short_long_ratio,rSteam_long,rsaltierthancrait_short,relderscrollsonline_short,rPolcompball_short,rontario_long
6,n_different_active_authors_link_id_link_id_short,n_different_active_authors_id_parent_id_medium,rCoronavirusDownunder_short,rbestoflegaladvice_short_long_ratio,rawfuleverything_long,rrant_short_long_ratio,rdankchristianmemes_long,rShitstatistssay_medium,rIncelTears_long,rfrance_short_long_ratio,rCCW_medium,rwoahdude_short_long_ratio,runitedkingdom_short,rSmite_short,rPOLITIC_short_long_ratio,rpcgaming_long,rSequelMemes_short,rKitchenConfidential_short,rPolcompball_short_long_ratio,rcanada_long
7,n_different_active_authors_link_id_link_id_long,n_different_comments_with_active_id_parent_id_...,rbestoflegaladvice_short_long_ratio,rMinneapolis_short_long_ratio,rchicago_short_long_ratio,rtoronto_short_long_ratio,rgaming_long,rGoldandBlack_long,rCoronavirusUS_short_long_ratio,rEnough_Sanders_Spam_short_long_ratio,rfrance_short_long_ratio,rflorida_short_long_ratio,rsoccer_

## Logistic Regression - statsmodelss

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
pca = PCA(n_components = 20)
# get the z-score
scaler = StandardScaler()

In [ ]:
np.random.seed(11)
msk = np.random.rand(len(df_lr)) < 0.75
data_train = df_lr.drop("duration", axis = 1)[msk]
data_test = df_lr.drop("duration", axis = 1)[~msk]

In [ ]:
X_train, y_train = data_train.drop("activation", axis = 1), data_train["activation"]
X_test, y_test = data_test.drop("activation", axis = 1), data_test["activation"]

In [ ]:
scaler.fit(X_train)
X_scale_train = scaler.transform(X_train)
# X_scale_train = X_train
# X_scale_train = scale_zscore(X_train)

In [ ]:
pca.fit(X_scale_train)
X_pca_train = pca.transform(X_scale_train)

In [ ]:
pd.DataFrame(X_pca_train, index = data_train.index).assign(y = y_train).columns

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       'y'],
      dtype='object')

In [ ]:
model = smf.logit(f"y ~ {'+'.join([f'col{u}' for u in range(20)])}", 
                  data = pd.DataFrame(X_pca_train, 
                                      index = data_train.index, 
                                      columns = [f"col{u}" for u in range(20)]).assign(y = y_train + 0.)).fit()

Optimization terminated successfully.
         Current function value: 0.597705
         Iterations 7


In [ ]:
pd.Series(np.dot(model.summary2().tables[1]["Coef."][1:], pca.components_), 
          index = X_train.columns).sort_values(ascending = False).head(10)



n_different_active_authors_parent_id_id_medium          0.041880
n_different_active_authors_parent_id_id_short           0.040649
n_different_active_authors_id_parent_id_medium          0.040052
n_different_active_authors_id_parent_id_short           0.038342
n_different_comments_with_active_parent_id_id_short     0.037038
n_different_comments_with_active_id_parent_id_medium    0.036676
n_different_comments_with_active_parent_id_id_medium    0.035792
n_different_comments_with_active_id_parent_id_short     0.032009
n_different_active_authors_id_parent_id_long            0.028815
n_different_comments_with_active_id_parent_id_long      0.028257
dtype: float64

## Survival Analysis

In [ ]:
import lifelines

In [ ]:
n_components = 20
cox = lifelines.CoxPHFitter()
pca = PCA(n_components = n_components)

In [ ]:
df_cox = df_lr.query("activation")

In [ ]:
X, y = df_cox.drop(["activation", "duration"], axis = 1), df_cox[["activation", "duration"]]
scaler.fit(X)
X_scale = scaler.transform(X)
pca.fit(X_scale)
X_pca = pca.transform(X_scale)    
    

In [ ]:
np.hstack([X_scale, y]).shape, X_scale.shape

((4141, 4003), (4141, 4001))

In [1]:
cox.fit(pd.DataFrame(np.hstack([X_pca, y]), columns = list(np.arange(n_components)) + ["activation", "duration"]),
        duration_col = "duration", 
        event_col = "activation", 
        formula = pd.DataFrame(X_pca, columns = np.arange(n_components)))

In [ ]:
cox.concordance_index_

0.5209472778104808

In [ ]:
cox.summary.loc[cox.summary["p"] < 0.05,"coef"]

covariate
1     0.007698
14    0.010745
Name: coef, dtype: float64

In [ ]:
original_coef_cox = pd.Series(np.dot(np.array(cox.summary.loc[cox.summary["p"] < 0.05,"coef"])[None,:], pca.components_[cox.summary["p"] < 0.05])[0], index = df_cox.drop(["activation", "duration"], axis = 1).columns)

In [ ]:
original_coef_cox.loc[[u for u in original_coef_cox.index if "norm" in u]].sort_values(ascending = False)

norm_climate_action_week_year_ratio      0.000166
norm_climate_action_week                 0.000148
norm_natural_disaster_week_year_ratio    0.000136
norm_climate_week_year_ratio             0.000127
norm_natural_disaster_week               0.000092
norm_climate_week                        0.000069
norm_climate_action_month                0.000033
norm_natural_disaster_month              0.000024
norm_climate_month                       0.000016
norm_climate_action_year                -0.000058
norm_natural_disaster_year              -0.000082
norm_climate_year                       -0.000083
dtype: float64

In [ ]:
original_coef_cox[[u+p for u in ft.features["control"] for p in ["_week", "_month", "_year"]]].sort_values(ascending = False)

n_comments_author_week_week        0.000883
n_different_subreddits_week        0.000876
n_different_subreddits_month       0.000802
n_comments_author_week_month       0.000762
n_different_subreddits_year        0.000529
n_comments_author_week_year        0.000519
n_active_days_author_week_month    0.000437
n_active_days_author_week_week     0.000419
n_active_days_author_week_year     0.000283
avg_comments_per_thread_month      0.000246
avg_comments_per_thread_week       0.000173
avg_comments_per_thread_year       0.000023
dtype: float64

In [ ]:
original_coef_cox[[u+p for u in ft.features["interaction"] for p in ["_week", "_month", "_year"]]].sort_values(ascending = False)

n_different_parent_id_with_active_week     0.000321
n_different_parent_id_with_active_month    0.000214
n_different_parent_id_with_active_year     0.000046
n_different_comments_with_active_week     -0.000033
n_different_active_authors_week           -0.000072
n_different_comments_with_active_year     -0.000108
n_different_comments_with_active_month    -0.000112
n_different_active_authors_month          -0.000252
n_different_active_authors_year           -0.000287
dtype: float64

In [ ]:
original_coef_cox.loc[[u for u in original_coef_cox.index if  u[0] == "r"]].sort_values(ascending = False).head(20)

relderscrollsonline_week                 0.001425
rKitchenConfidential_week_year_ratio     0.001266
rwallstreetbetsHUZZAH_week               0.001138
relderscrollsonline_week_year_ratio      0.001138
rchess_week_year_ratio                   0.001138
rCoronavirusDownunder_week               0.001138
rwallstreetbetsHUZZAH_week_year_ratio    0.001138
rSuperstonk_week_year_ratio              0.000921
rMLPLounge_month                         0.000919
r49ers_week                              0.000913
rmoviescirclejerk_week_year_ratio        0.000913
rMLPLounge_week                          0.000887
rmylittlepony_week                       0.000885
rSmite_month                             0.000877
rGuildwars2_week_year_ratio              0.000877
rGuildwars2_week                         0.000877
rMLPLounge_year                          0.000865
rDebateReligion_week_year_ratio          0.000808
rGreenBayPackers_month                   0.000770
rbtc_week_year_ratio                     0.000754


In [ ]:
pd.Series(pca.components_[2,:], index = df_cox.drop(["activation", "duration"], axis = 1).columns).sort_values()

quartile1_affluence                     -0.101507
quartile1_age                           -0.086113
rokbuddyretard_year                     -0.083252
rTheRightCantMeme_year                  -0.081293
rToiletPaperUSA_year                    -0.079848
                                           ...   
rCoronavirusDownunder_week               0.058494
rwallstreetbetsHUZZAH_week_year_ratio    0.058494
rchess_week_year_ratio                   0.058494
rwallstreetbetsHUZZAH_week               0.058494
relderscrollsonline_week_year_ratio      0.058494
Length: 4001, dtype: float64

### Bootstrap

In [ ]:
def pca_cox(df_cox, features = None):
    n_components = 8
    cox = lifelines.CoxPHFitter()
    pca = PCA(n_components = n_components)
    df_cox = df_cox.query("activation")
    X, y = df_cox.drop(["activation", "duration"], axis = 1) + 0., df_cox[["activation", "duration"]] + 0.
    
    if features is None:
        features = list(df_cox.drop(["activation", "duration"], axis = 1).columns)
    X = X[features]
    scaler.fit(X)
    X_scale = scaler.transform(X)
    pca.fit(X_scale)
    X_pca = pca.transform(X_scale)    
        
    cox.fit(pd.DataFrame(np.hstack([X_pca, y]), columns = list(np.arange(n_components)) + ["activation", "duration"]),
            duration_col = "duration", 
            event_col = "activation", 
            formula = pd.DataFrame(X_pca, columns = np.arange(n_components)))
    original_coef_cox = pd.Series(np.dot(np.array(cox.summary.loc[cox.summary["p"] < 0.05,"coef"])[None,:], 
                                         pca.components_[cox.summary["p"] < 0.05])[0], 
                                         index = features)

    return original_coef_cox

In [2]:
n_bootstraps = 20
coef_boot = pd.DataFrame([pca_cox(cle.get_bootstrap_df(df_cox, random_state = k)) for k in range(n_bootstraps)])

In [100]:
pvalues = pd.DataFrame([(coef_boot >= 0).mean(), 1-(coef_boot > 0).mean()]).min()

In [101]:
coef_cox = pd.DataFrame({"coef": coef_boot.mean(), "p":pvalues})

In [114]:
coef_boot.loc[:,((coef_boot >= 0).mean() < 0.15)|((coef_boot >= 0).mean() > 0.85)].mean().sort_values(ascending = False).head(40)

n_different_parent_id_with_active_week    0.000904
n_different_comments_with_active_week     0.000658
n_active_days_author_week_week            0.000623
rBlackPeopleTwitter_week                  0.000621
rtechnews_month                           0.000546
rtechnology_month                         0.000525
rcenterleftpolitics_week                  0.000506
rIAmA_month                               0.000498
rThe_Mueller_week                         0.000490
rPolitical_Revolution_week                0.000483
rcenterleftpolitics_week_year_ratio       0.000482
rworldpolitics_month                      0.000475
rAstuff_week_year_ratio                   0.000468
rAnythingGoesNews_week_year_ratio         0.000468
rColumbus_week_year_ratio                 0.000468
rAstuff_week                              0.000468
rEconomics_month                          0.000457
rPolitical_Revolution_week_year_ratio     0.000451
rtechnology_week                          0.000430
rEnough_Sanders_Spam_week      